In [62]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatText
from IPython.display import clear_output


from scipy.integrate import solve_ivp


In [71]:

def asymmetric_dual_hump_potential(x, k1=1.0, k2=1.0, a1=1.0, a2=1.0):
    """Asymmetric potential with two humps of maxima separated by distance."""
    potential = 0.5 * k1 * (x**2 - a1)**2 + 0.5 * k2 * (x**2 - a2)**2
    force = -2 * k1 * x * (x**2 - a1) - 2 * k2 * x * (x**2 - a2)
    return potential, force

# Simulate particle motion
def simulate_motion(potential_func, params, x0=1.0, v0=0.0, t_min=0, t_max=3600, dt=0.01):
    def dynamics(t, y):
        x, v = y
        _, force = potential_func(x, **params)
        return [v, force]

    t_span = (t_min, t_max)
    t_eval = np.arange(t_min, t_max, dt)
    sol = solve_ivp(dynamics, t_span, [x0, v0], t_eval=t_eval)
    return sol.t, sol.y[0], sol.y[1]

# Plot the potential and phase space
def plot_potential_phase_space(k1=1.0, k2=1.0, a1=1.0, a2=1.0):

    clear_output(wait=True)
    x = np.linspace(-20, 20, 500)
    
    potential, _ = asymmetric_dual_hump_potential(x, k1, k2, a1, a2)
    params = {'k1': k1, 'k2': k2, 'a1': a1, 'a2': a2}
    potential_func = asymmetric_dual_hump_potential
    
    # Simulate motion
    t, x_vals, v_vals = simulate_motion(potential_func, params, t_max=360)
    print(len(x_vals), len(v_vals))
    # Plotting
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot potential
    ax[0].plot(x, np.log(potential), label="Asymmetric Dual Hump Potential")
    ax[0].set_xlabel("x")
    ax[0].set_ylabel("Potential (log scale)")
    ax[0].set_title("Potential")
    ax[0].grid(True)
    ax[0].legend()

    # Plot phase space
    ax[1].plot(x_vals, v_vals, label="Phase Space")
    ax[1].set_xlabel("x")
    ax[1].set_ylabel("v")
    ax[1].set_title("Phase Space")
    ax[1].grid(True)
    ax[1].legend()

    plt.tight_layout()
    plt.show()

# Interactive input boxes for parameters



In [72]:
interact(
    plot_potential_phase_space,
    k1=FloatText(value=1.0, description="k1 (Hump 1)"),
    k2=FloatText(value=1.0, description="k2 (Hump 2)"),
    a1=FloatText(value=1.0, description="a1 (Hump 1)"),
    a2=FloatText(value=1.0, description="a2 (Hump 2)"),
)

interactive(children=(FloatText(value=1.0, description='k1 (Hump 1)'), FloatText(value=1.0, description='k2 (H…

<function __main__.plot_potential_phase_space(k1=1.0, k2=1.0, a1=1.0, a2=1.0)>